# Video-Model for B-Chef

In this notebool we will ~~try to~~ train own version of pre-trained model, called SlowFast

## Imports and defininigs

In [2]:
!pip install torch torchvision torchaudio pytorchvideo --quiet
!pip install omegaconf einops tqdm --quiet
!apt install p7zip-full --quiet

import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━

In [25]:
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder

In [16]:
!pip install mlflow --quiet
import mlflow
import mlflow.pytorch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.8/749.8 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
BASE_DIR = "/kaggle/input/b-chef-tensors"

# CSV with metadata
TRAIN_CSV = os.path.join(BASE_DIR, "train.csv")
VAL_CSV   = os.path.join(BASE_DIR, "val.csv")
TEST_CSV  = os.path.join(BASE_DIR, "test.csv")
LABELS_SUMMARY = os.path.join(BASE_DIR, "labels_summary.csv")

# folders with tesors
TENSORS_DIR = os.path.join(BASE_DIR, "tensors", "tensors")
TRAIN_DIR = os.path.join(TENSORS_DIR, "train")
VAL_DIR   = os.path.join(TENSORS_DIR, "val")
TEST_DIR  = os.path.join(TENSORS_DIR, "test")

# check
print("Train tensors:", os.listdir(TRAIN_DIR)[:5])
print("Val tensors:", os.listdir(VAL_DIR)[:5])
print("Test tensors:", os.listdir(TEST_DIR)[:5])
print("Train CSV exists:", os.path.exists(TRAIN_CSV))
print("Labels summary exists:", os.path.exists(LABELS_SUMMARY))


Train tensors: ['194_cut_slices.pt', '56_peel.pt', '241_peel.pt', '14_put_on_cutting-board.pt', '359_stir.pt']
Val tensors: ['65_put_on_plate.pt', '87_cut_slices.pt', '14_put_on_cutting-board.pt', '86_pour.pt', '49_cut_apart.pt']
Test tensors: ['14_cut_apart.pt', '31_pour.pt', '56_peel.pt', '11_peel.pt', '92_pour.pt']
Train CSV exists: True
Labels summary exists: True


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Work wth data

### Check that everything is accessible

In [4]:
train_df = pd.read_csv(TRAIN_CSV)
val_df   = pd.read_csv(VAL_CSV)
test_df  = pd.read_csv(TEST_CSV)
labels_summary = pd.read_csv(LABELS_SUMMARY)

print("Train CSV sample:")
display(train_df.head())

print("Labels summary:")
display(labels_summary)

Train CSV sample:


,filename,label
0,s10-d02-cam-002_23807_23844.avi,put_in_bowl
1,s10-d02-cam-002_7280_8569.avi,peel
2,s12-d10-cam-002_9017_9073.avi,cut_apart
3,s11-d12-cam-002_9131_9323.avi,put_on_plate
4,s16-d06-cam-002_9363_9547.avi,pour


Labels summary:


,label,count
0,cut_apart,75
1,cut_dice,75
2,cut_slices,74
3,grate,5
4,mix,8
5,open_egg,14
6,peel,75
7,pour,75
8,put_in_bowl,73
9,put_in_pan-pot,58


In [7]:
sample_file = os.listdir(TRAIN_DIR)[0]
sample_path = os.path.join(TRAIN_DIR, sample_file)
print("Loading:", sample_file)

tensor, label = torch.load(sample_path)
print("Tensor shape:", tensor.shape)
print("Label:", label)
# 32 frames, 3 channels (RGB), each frame size 224x224

Loading: 194_cut_slices.pt
Tensor shape: torch.Size([32, 3, 224, 224])
Label: cut_slices


### Creating TensorVideoDataset

It is class that basically do the cell above for all files, but more convenient

In [43]:
class TensorVideoDataset(Dataset):
    def __init__(self, tensor_dir, csv_path, encoder=None, fit_encoder=False):
        self.tensor_dir = tensor_dir
        self.files = [f for f in os.listdir(tensor_dir) if f.endswith(".pt")]
        self.df = pd.read_csv(csv_path) if csv_path else None
        self.encoder = encoder
        if fit_encoder:
            labels = [f.split("_", 1)[1].replace(".pt", "") for f in self.files]
            self.encoder = LabelEncoder()
            self.encoder.fit(labels)


    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.tensor_dir, self.files[idx])
        video_tensor, _ = torch.load(path)

        label_name = os.path.basename(path).split("_", 1)[1].replace(".pt", "")

        if self.encoder:
            label = torch.tensor(self.encoder.transform([label_name])[0])
        else:
            label = label_name

        return video_tensor, label


In [44]:
# creating datasets
train_dataset = TensorVideoDataset(
    tensor_dir=TRAIN_DIR,
    csv_path=TRAIN_CSV,
    fit_encoder=True
)

val_dataset = TensorVideoDataset(
    tensor_dir=VAL_DIR,
    csv_path=VAL_CSV,
    encoder=train_dataset.encoder  # use same encoder
)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples:   {len(val_dataset)}")


Train samples: 529
Val samples:   114


### Create dataloaders

For batches, shuffling & iterations

In [60]:
def pad_collate(batch):
    # batch = [(video_tensor, label), ...]

    max_len = max(v.shape[0] for v, _ in batch)
    
    videos = []
    labels = []
    
    for v, l in batch:
        if v.shape[0] < max_len:
            pad = torch.zeros((max_len - v.shape[0], *v.shape[1:]))
            v = torch.cat([v, pad], dim=0)
        videos.append(v)
        labels.append(l)
    
    videos = torch.stack(videos)
    labels = torch.tensor(labels)
    
    return videos, labels


In [61]:
BATCH_SIZE = 8 if device.type == "cuda" else 2
print("Batch size set to:", BATCH_SIZE)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)

sample_batch = next(iter(train_loader))
videos, labels = sample_batch
print(f"Videos batch shape: {videos.shape}")
print(f"Labels batch shape: {labels.shape}")
print(f"Labels: {labels}")


Batch size set to: 2
Videos batch shape: torch.Size([2, 32, 3, 224, 224])
Labels batch shape: torch.Size([2])
Labels: tensor([ 8, 11])


## Model

Begin with simple, straigh-forward version

In [62]:
import torch
import torch.nn as nn

class SlowFastModel(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.num_classes = num_classes

        # Slow path: low frame rate, extracts context
        self.slow_path = nn.Sequential(
            nn.Conv3d(3, 32, kernel_size=3, stride=(1,2,2), padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((4, 14, 14))
        )

        # Fast path: high frame rate, extracts dynamics
        self.fast_path = nn.Sequential(
            nn.Conv3d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((4, 14, 14))
        )

        self.fc = nn.Sequential(
            nn.Linear((32 + 8) * 4 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: [B, T, C, H, W] -> [B, C, T, H, W]
        x = x.permute(0, 2, 1, 3, 4)

        # divide into slow and fast flows
        slow = x[:, :, ::8, :, :]  # take 1/8 frames
        fast = x                     # all frames

        # Conv3d + AdaptiveAvgPool3d
        slow_feat = self.slow_path(slow)
        fast_feat = self.fast_path(fast)

        # "flattening" tensors for fully con layer
        slow_feat = slow_feat.flatten(start_dim=1)
        fast_feat = fast_feat.flatten(start_dim=1)

        out = torch.cat([slow_feat, fast_feat], dim=1)

        return self.fc(out)


In [63]:
mlflow.set_tracking_uri("file:///kaggle/working/mlruns")
mlflow.set_experiment("b-chef-slowfast")


<Experiment: artifact_location='file:///kaggle/working/mlruns/704353735472346475', creation_time=1760983364287, experiment_id='704353735472346475', last_update_time=1760983364287, lifecycle_stage='active', name='b-chef-slowfast', tags={}>

In [64]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    loop = tqdm(dataloader, desc=f"🧠 Train Epoch {epoch+1}", leave=False)
    for inputs, labels in loop:
        inputs = inputs.to(device)
        labels = labels.long().to(device)


        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    train_loss = running_loss / len(dataloader)
    train_acc = 100 * correct / total

    mlflow.log_metric("train_loss", train_loss, step=epoch)
    mlflow.log_metric("train_acc", train_acc, step=epoch)

    return train_loss, train_acc


@torch.no_grad()
def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    loop = tqdm(dataloader, desc=f"🔍 Val Epoch {epoch+1}", leave=False)
    for inputs, labels in loop:
        inputs = inputs.to(device)
        labels = labels.long().to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    val_loss /= len(dataloader)
    val_acc = 100 * correct / total

    mlflow.log_metric("val_loss", val_loss, step=epoch)
    mlflow.log_metric("val_acc", val_acc, step=epoch)

    return val_loss, val_acc


In [65]:
num_classes = 13
model = SlowFastModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 5

with mlflow.start_run(run_name="slowfast_v1"):
    mlflow.log_params({
        "num_classes": num_classes,
        "lr": 1e-4,
        "epochs": num_epochs,
        "optimizer": "Adam",
        "batch_size": train_loader.batch_size
    })
    
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\n🌍 Epoch [{epoch+1}/{num_epochs}]")
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
        val_loss, val_acc = validate(model, val_loader, criterion, device, epoch)

        print(f"📈 Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"🧪 Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            mlflow.pytorch.log_model(model, artifact_path="best_model")
            print("💾 Best model updated!")

    mlflow.log_metric("best_val_acc", best_val_acc)



🌍 Epoch [1/5]


2025/10/20 19:54:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


📈 Train Loss: 2.5263 | Train Acc: 15.50%
🧪 Val   Loss: 2.1605 | Val   Acc: 26.32%


2025/10/20 19:54:17 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/20 19:54:40 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/20 19:54:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


💾 Best model updated!

🌍 Epoch [2/5]


...

2025/10/20 20:15:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


📈 Train Loss: 1.7931 | Train Acc: 39.51%
🧪 Val   Loss: 1.7353 | Val   Acc: 47.37%


2025/10/20 20:15:41 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/20 20:15:58 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/20 20:15:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


💾 Best model updated!

🌍 Epoch [5/5]


📈 Train Loss: 1.6951 | Train Acc: 44.42%
🧪 Val   Loss: 1.6936 | Val   Acc: 42.98%
